In [1]:
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv()

True

# Prompt+LLM
基本构成： PromptTemplate / ChatPromptTemplate -> LLM / ChatModel -> OutputParser

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("给我讲一个关于{foo}的笑话")
model = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
)
chain = prompt | model

In [3]:
chain.invoke({"foo": "狗熊"})

AIMessage(content='好的，这里有一个关于狗熊的笑话：\n\n为什么狗熊不喜欢在雨天出去玩耍？\n\n因为它怕被淋湿，变成“湿熊”啦！哈哈哈！', response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 23, 'total_tokens': 97, 'pre_token_count': 16384, 'pre_total': 124, 'adjust_total': 123, 'final_total': 1}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a7a3dc6b-cc9a-4f5d-8e2e-ee127db11597-0')

# 自定义停止输出符

In [5]:
chain = prompt | model.bind(stop=["\n"])

In [6]:
chain.invoke({"foo": "狗熊"})

AIMessage(content='好的，这里有一个关于狗熊的笑话：', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 23, 'total_tokens': 42, 'pre_token_count': 16384, 'pre_total': 124, 'adjust_total': 123, 'final_total': 1}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-18bffc26-2004-4447-b020-ce40a0297970-0')

In [7]:
functions = [
    {
        "name": "joke",
        "description": "讲笑话",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "笑话的开头"},
                "punchline": {
                    "type": "string",
                    "description": "爆梗的结尾",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]
chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions)

# 兼容openai函数调用的方式

In [8]:
chain.invoke({"foo": "男人"}, config={})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"setup":"为什么男人喜欢开车？","punchline":"因为开车时他们可以控制方向盘，而在家里他们控制不了任何东西。"}', 'name': 'joke'}}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 90, 'total_tokens': 144, 'pre_token_count': 16384, 'pre_total': 124, 'adjust_total': 123, 'final_total': 1}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-76441a98-ea88-4806-b059-0b0f949394b9-0')

# 输出解析器

In [9]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()

In [10]:
chain.invoke({"foo": "女人"})

'为什么女人喜欢购物？\n\n因为购物可以让她们忘记所有烦恼，直到看到信用卡账单的时候。'

# 与函数调用混合使用


In [11]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonOutputFunctionsParser()
)

In [12]:
chain.invoke({"foo": "女人"})

{'setup': '为什么女人喜欢逛街？', 'punchline': '因为逛街可以让她们忘记自己的烦恼，直到刷爆信用卡的时候。'}

In [13]:
#只输出setup
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

chain = (
    {"foo": RunnablePassthrough()} #使用RunnablePassthrough()跳过prompt
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="punchline") # 定义输出的key
)

In [16]:
chain.invoke({"foo": "女人"})
#使用RunnablePassthrough()跳过prompt


'因为逛街可以让她们忘记自己的烦恼，直到刷爆信用卡的时候。'

In [17]:
chain.invoke("女人")

'因为逛街可以让她们找到自己的幸福感！'

# 使用Runnables来连接多链结构

In [18]:
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv()

True

In [26]:
from operator import itemgetter #获取可迭代对象中指定索引或键对应的元素

from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt1 = ChatPromptTemplate.from_template("{person}来自于哪个城市?")

prompt2 = ChatPromptTemplate.from_template(
    "{city}属于哪个市? 用{language}来回答"
)

model = ChatOpenAI()

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")} #获取invoke中的language
    | prompt2
    | model
    | StrOutputParser()
)
# chain1.invoke({"person": "马化腾"})
chain2.invoke({"person": "马化腾", "language": "中文"})

'汕头市。'

In [24]:
chain1.invoke({"person": "马化腾"})

'马化腾（Pony Ma）来自中国广东省汕头市。'

In [29]:
chain2.invoke({"person": "马化腾", "language": "中文"})

'汕头市。'

In [30]:
from langchain_core.runnables import RunnablePassthrough

prompt1 = ChatPromptTemplate.from_template(
    "生成一个{attribute}属性的颜色。除了返回这个颜色的名字不要做其他事:"
)
prompt2 = ChatPromptTemplate.from_template(
    "什么水果是这个颜色:{color},只返回这个水果的名字不要做其他事情:"
)
prompt3 = ChatPromptTemplate.from_template(
    "哪个国家的国旗有这个颜色:{color},只返回这个国家的名字不要做其他事情:"
)
prompt4 = ChatPromptTemplate.from_template(
    "有这个颜色的水果是{fruit},有这个颜色的国旗是{country}？"
)

model_parser = model | StrOutputParser()
# 生成一个颜色
color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser
question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)

In [36]:
question_generator.invoke("蓝色")

ChatPromptValue(messages=[HumanMessage(content='有这个颜色的水果是蓝莓。,有这个颜色的国旗是希腊？')])

# 多链执行与结果合并
      输入
      / \
     /   \
 分支1   分支2
     \   /
      \ /
    合并结果

# 唯物辩证链

In [37]:
planner = (
    ChatPromptTemplate.from_template("生成一个关于{input}的论点")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "列出以下内容的优点或积极方面:{base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "列出以下内容的缺点或消极方面:{base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "积极:\n{results_1}\n\n消极:\n{results_2}"),
            ("system", "根据评论生成最终的回复"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [38]:
chain.invoke({"input": "生孩子"})

'生孩子是一个复杂而深刻的经历，既有积极的一面，也有消极的挑战。积极方面包括体验到无比的爱和责任感，延续家族血脉，获得成就感和幸福快乐，这些都是值得珍惜的。然而，生育孩子也伴随着巨大的身心压力，需要投入大量时间精力金钱，可能会影响个人事业和生活质量，以及面临教养教育的挑战和经济压力。重要的是，每个人在决定生育孩子时都应该认真考虑并准备好迎接这些挑战，以确保为孩子提供一个健康快乐的成长环境。'

# 查询SQL

In [40]:
from langchain_core.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

In [41]:
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [42]:
db.get_table_info()

'\nCREATE TABLE "Album" (\n\t"AlbumId" INTEGER NOT NULL, \n\t"Title" NVARCHAR(160) NOT NULL, \n\t"ArtistId" INTEGER NOT NULL, \n\tPRIMARY KEY ("AlbumId"), \n\tFOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")\n)\n\n/*\n3 rows from Album table:\nAlbumId\tTitle\tArtistId\n1\tFor Those About To Rock We Salute You\t1\n2\tBalls to the Wall\t2\n3\tRestless and Wild\t2\n*/\n\n\nCREATE TABLE "Artist" (\n\t"ArtistId" INTEGER NOT NULL, \n\t"Name" NVARCHAR(120), \n\tPRIMARY KEY ("ArtistId")\n)\n\n/*\n3 rows from Artist table:\nArtistId\tName\n1\tAC/DC\n2\tAccept\n3\tAerosmith\n*/\n\n\nCREATE TABLE "Customer" (\n\t"CustomerId" INTEGER NOT NULL, \n\t"FirstName" NVARCHAR(40) NOT NULL, \n\t"LastName" NVARCHAR(20) NOT NULL, \n\t"Company" NVARCHAR(80), \n\t"Address" NVARCHAR(70), \n\t"City" NVARCHAR(40), \n\t"State" NVARCHAR(40), \n\t"Country" NVARCHAR(40), \n\t"PostalCode" NVARCHAR(10), \n\t"Phone" NVARCHAR(24), \n\t"Fax" NVARCHAR(24), \n\t"Email" NVARCHAR(60) NOT NULL, \n\t"SupportRepId" INTEG

In [43]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
)

def get_schema(_):
    return db.get_table_info()



sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | model.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [44]:
sql_response.invoke({"question": "How many artists are there?"})

'SELECT COUNT(*) AS TotalArtists\nFROM Artist;'

In [45]:
def run_query(query):
    print(query)
    return db.run(query)

template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

full_chain = (
    RunnablePassthrough.assign(query=sql_response).assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"]),
    )
    | prompt_response
    | model
)

In [47]:
full_chain.invoke({"question": "How many artists are there?"})

AIMessage(content='There are a total of 275 artists in the database.', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 2154, 'total_tokens': 2166, 'pre_token_count': 16384, 'pre_total': 140, 'adjust_total': 134, 'final_total': 6}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-827ecaf5-904e-43d9-a1f3-73637720c920-0')

# 自定义输出解析器
- python编程助手

In [55]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
)
from langchain_experimental.utilities import PythonREPL
from langchain_openai import ChatOpenAI

template = """根据用户需求帮助用户编写python代码. 

只需要返回makedown格式的python代码, 比如:

```python
....
```"""
prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])

model = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
)

#自定义输出解析，只返回python代码
def _sanitize_output(text: str):
    _, after = text.split("```python")
    return after.split("```")[0]

#定义链
chain = prompt | model | StrOutputParser() | _sanitize_output | PythonREPL().run

In [51]:
chain.invoke({"input": "从10里面循环找出所有质数"})

Python REPL can execute arbitrary code. Use with caution.


'[2, 3, 5, 7]\n'

In [56]:
chain.invoke({"input": "python实现Epoll事件循环"})

'\nimport select\n\nepoll = select.epoll()\n\nwhile True:\n    events = epoll.poll()\n    for fileno, event in events:\n        # 处理事件\n'